# 2. Query Database SQLite

We will using `chinook sample database` from **sqlitetutorial.net**

Database Source & Reference: [https://www.sqlitetutorial.net/sqlite-sample-database/](https://www.sqlitetutorial.net/sqlite-sample-database/)

There are 11 tables in the chinook sample database.

* **`employees`** table stores employees data such as employee id, last name, first name, etc. It also has a field named **`ReportsTo`** to specify who reports to whom.
* **`customers`** table stores customers data.
* **`invoices`** & **`invoice_items`** tables: these two tables store invoice data. <br>The **`invoices`** table stores invoice header data and the **`invoice_items`** table stores the invoice line items data.
* **`artists`** table stores artists data. It is a simple table that contains only the artist id and name.
* **`albums`** table stores data about a list of tracks. Each album belongs to one artist. However, one artist may have multiple albums.
* **`media_types`** table stores media types such as MPEG audio and AAC audio files.
* **`genres`** table stores music types such as rock, jazz, metal, etc.
* **`tracks`** table stores the data of songs. Each track belongs to one album.
* **`playlists`** & **`playlist_track`** tables: playlists table store data about playlists. Each playlist contains a list of tracks. Each track may belong to multiple playlists. The relationship between the **`playlists`** table and **`tracks`** table is many-to-many. The **`playlist_track`** table is used to reflect this relationship.

<br>

<img src="database/chinook_schema.png" width="800">

In [1]:
# Import SQLite, Create connection and create CursorObject
import sqlite3
connection = sqlite3.connect('database/chinook.db')
cursorObj = connection.cursor()

### Show All Tables from chinook Database

In [2]:
cursorObj.execute("""SELECT name FROM sqlite_master 
                     WHERE type='table' AND name NOT LIKE 'sqlite_%'""").fetchall()

[('albums',),
 ('artists',),
 ('customers',),
 ('employees',),
 ('genres',),
 ('invoices',),
 ('invoice_items',),
 ('media_types',),
 ('playlists',),
 ('playlist_track',),
 ('tracks',)]

### Show Columns Metadata from table

In [3]:
# Show Columns Name from table `tracks`
cursorObj.execute("PRAGMA table_info(tracks)").fetchall()

[(0, 'TrackId', 'INTEGER', 1, None, 1),
 (1, 'Name', 'NVARCHAR(200)', 1, None, 0),
 (2, 'AlbumId', 'INTEGER', 0, None, 0),
 (3, 'MediaTypeId', 'INTEGER', 1, None, 0),
 (4, 'GenreId', 'INTEGER', 0, None, 0),
 (5, 'Composer', 'NVARCHAR(220)', 0, None, 0),
 (6, 'Milliseconds', 'INTEGER', 1, None, 0),
 (7, 'Bytes', 'INTEGER', 0, None, 0),
 (8, 'UnitPrice', 'NUMERIC(10,2)', 1, None, 0)]

<br>

## Query Database

### Execute Query using `cursorObject`

In [4]:
cursorObj.execute("SELECT * FROM artists LIMIT 5")
cursorObj.fetchall()

[(1, 'AC/DC'),
 (2, 'Accept'),
 (3, 'Aerosmith'),
 (4, 'Alanis Morissette'),
 (5, 'Alice In Chains')]

### Query using `cursorObject` OneLine using `fetchall()`

In [5]:
cursorObj.execute("SELECT * FROM albums LIMIT 5").fetchall()

[(1, 'For Those About To Rock We Salute You', 1),
 (2, 'Balls to the Wall', 2),
 (3, 'Restless and Wild', 2),
 (4, 'Let There Be Rock', 1),
 (5, 'Big Ones', 3)]

### Create Query Function

In [6]:
def query_fetch(query, cursorObject=cursorObj):
    """
    Execute Query and fetch all return data
    ----------------------------------------
    Params: 
    query: SQL Query Statement
    cursorObject: Default cursorObj"""
    cursorObject.execute(query)
    return cursorObject.fetchall()

In [7]:
query = """SELECT artists.Name , albums.Title
           FROM artists
           INNER JOIN albums
           ON albums.ArtistId=artists.ArtistId
           WHERE artists.Name LIKE 'Queen'"""

query_fetch(query)

[('Queen', 'Greatest Hits II'),
 ('Queen', 'Greatest Hits I'),
 ('Queen', 'News Of The World')]

<br>

### Query With Columns Name

In [8]:
def usecols_query(query, cursorObject=cursorObj):
    """
    Execute Query and fetch all and columns as return data
    -----------------------------------------------------
    Params: 
    query: SQL Query Statement
    cursorObject: Default cursorObj"""
    cursorObject.execute(query)
    col_list = [tuple_col[0] for tuple_col in cursorObject.description]
    return col_list, cursorObject.fetchall()

In [9]:
query = """SELECT artists.Name AS 'Artist Name', COUNT(albums.Title) AS 'Total Albums'
           FROM artists
           INNER JOIN albums
           ON albums.ArtistId=artists.ArtistId
           GROUP BY artists.Name
           ORDER BY `Total Albums` DESC
           LIMIT 7
           """

usecols_query(query)

(['Artist Name', 'Total Albums'],
 [('Iron Maiden', 21),
  ('Led Zeppelin', 14),
  ('Deep Purple', 11),
  ('U2', 10),
  ('Metallica', 10),
  ('Ozzy Osbourne', 6),
  ('Pearl Jam', 5)])

#### Unpacking Function

In [10]:
cols, data = usecols_query(query)
print("Columns Name: \n", cols)
print("\nData:")
for row in data:
    print(row)

Columns Name: 
 ['Artist Name', 'Total Albums']

Data:
('Iron Maiden', 21)
('Led Zeppelin', 14)
('Deep Purple', 11)
('U2', 10)
('Metallica', 10)
('Ozzy Osbourne', 6)
('Pearl Jam', 5)


<br>

### Returns Pandas DataFrame

In [11]:
# Import pandas
import pandas as pd

def dataframe_query(query, cursorObject=cursorObj, pandas=pd):
    """
    Execute Query and fetch all data and return dataframe
    -----------------------------------------------------
    Params: 
    query: SQL Query Statement
    cursorObject: Default cursorObj
    pandas: Imported Pandas default=pd"""
    cursorObject.execute(query)
    col_list = [tuple_col[0] for tuple_col in cursorObject.description]
    return pd.DataFrame(cursorObject.fetchall(), columns=col_list)

In [12]:
query = """SELECT genres.Name AS 'Genre Name', albums.Title AS 'Album Title', 
           tracks.Name AS 'Track Name'
           FROM tracks
           INNER JOIN genres ON genres.GenreId = tracks.GenreId
           INNER JOIN albums ON albums.AlbumId = tracks.AlbumId"""

dataframe_query(query)

,Genre Name,Album Title,Track Name
0,Rock,For Those About To Rock We Salute You,For Those About To Rock (We Salute You)
1,Rock,Balls to the Wall,Balls to the Wall
2,Rock,Restless and Wild,Fast As a Shark
3,Rock,Restless and Wild,Restless and Wild
4,Rock,Restless and Wild,Princess of the Dawn
...,...,...,...
3498,Classical,Respighi:Pines of Rome,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...
3499,Classical,Schubert: The Late String Quartets & String Qu...,"String Quartet No. 12 in C Minor, D. 703 ""Quar..."
3500,Classical,Monteverdi: L'Orfeo,"L'orfeo, Act 3, Sinfonia (Orchestra)"
3501,Classical,Mozart: Chamber Music,"Quintet for Horn, Violin, 2 Violas, and Cello ..."


<br><br>

## Query Directly using Pandas

Pandas has function **`read_sql_query`** which takes query argument and **`connection Object`**, and it will automatically return a dataframe.

In [13]:
import pandas as pd

In [14]:
query = """SELECT genres.Name AS 'Genre Name', albums.Title AS 'Album Title', 
           tracks.Name AS 'Track Name'
           FROM tracks
           INNER JOIN genres ON genres.GenreId = tracks.GenreId
           INNER JOIN albums ON albums.AlbumId = tracks.AlbumId"""

df = pd.read_sql_query(query, con=connection)
df

,Genre Name,Album Title,Track Name
0,Rock,For Those About To Rock We Salute You,For Those About To Rock (We Salute You)
1,Rock,Balls to the Wall,Balls to the Wall
2,Rock,Restless and Wild,Fast As a Shark
3,Rock,Restless and Wild,Restless and Wild
4,Rock,Restless and Wild,Princess of the Dawn
...,...,...,...
3498,Classical,Respighi:Pines of Rome,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...
3499,Classical,Schubert: The Late String Quartets & String Qu...,"String Quartet No. 12 in C Minor, D. 703 ""Quar..."
3500,Classical,Monteverdi: L'Orfeo,"L'orfeo, Act 3, Sinfonia (Orchestra)"
3501,Classical,Mozart: Chamber Music,"Quintet for Horn, Violin, 2 Violas, and Cello ..."


In [15]:
type(df)

pandas.core.frame.DataFrame